In [1]:
import pandas as pd


In [2]:
data = pd.read_csv("fuel.csv")

<ipython-input-2-b3a57def2abe>:1: DtypeWarning: Columns (7,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("fuel.csv")


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 81 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [4]:
data = data.dropna(axis=1,how='all') # Cancellazione delle colonne nulle

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 73 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [6]:
data.columns[data.isnull().any()] #Colonne con valori nulli

Index(['drive', 'transmission', 'transmission_type', 'engine_descriptor',
       'engine_cylinders', 'engine_displacement', 'turbocharger',
       'gas_guzzler_tax', 'alternative_fuel_technology'],
      dtype='object')

In [7]:
data = data.drop('gas_guzzler_tax', axis=1) #Tassa americana, non serve


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 72 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [9]:
data.convert_dtypes() #Regolarizza tutti i NAN nel formato più appropriato al resto della colonna (0, ' ', 0.0)

,vehicle_id,year,make,model,class,drive,transmission,transmission_type,engine_index,engine_descriptor,...,hours_to_charge_240v,hours_to_charge_ac_240v,composite_city_mpg,composite_highway_mpg,composite_combined_mpg,range_ft1,city_range_ft1,highway_range_ft1,city_range_ft2,highway_range_ft2
0,26587,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,<NA>,Manual 5-Speed,<NA>,9001,(FFS),...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
1,27705,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,<NA>,Manual 5-Speed,<NA>,9005,(FFS) CA model,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
2,26561,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,<NA>,Manual 5-Speed,<NA>,9002,(FFS),...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
3,27681,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,<NA>,Manual 5-Speed,<NA>,9006,(FFS) CA model,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
4,27550,1984,AM General,DJ Po Vehicle 2WD,Special Purpose Vehicle 2WD,2-Wheel Drive,Automatic 3-Speed,<NA>,1830,(FFS),...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38108,37564,2017,Volvo,XC60 FWD,Small Sport Utility Vehicle 2WD,Front-Wheel Drive,Automatic (S8),<NA>,90,SIDI,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
38109,37547,2017,Volvo,XC90 AWD,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),<NA>,52,SIDI,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
38110,37548,2017,Volvo,XC90 AWD,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),<NA>,53,SIDI,...,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
38111,37703,2017,Volvo,XC90 AWD PHEV,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),<NA>,54,SIDI; PHEV,...,3.0,0.0,29,32,30,0,0.0,0.0,13.84,13.3


In [10]:
data.info() #Finale

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 72 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [11]:
data.drop(list(data.filter(regex = 'unrounded')), axis = 1, inplace = True) #Valori unrounded non sono utilizzati, in quanto mancano per la maggioranza delle entry ed il livello di approssimazione dei dati rounded è accettabile.


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 66 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [13]:
data.astype(bool).sum(axis=0)



vehicle_id           38113
year                 38113
make                 38113
model                38113
class                38113
                     ...  
range_ft1              133
city_range_ft1         106
highway_range_ft1      106
city_range_ft2          65
highway_range_ft2       65
Length: 66, dtype: int64

In [14]:
data.drop(list(data.filter(regex = 'range')), axis = 1, inplace = True) #Valori range non sono utilizzati, in quanto mancano per la maggioranza delle entry ed il tipo di dato non è correlato all'analisi

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 61 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [16]:
data.astype(bool).sum(axis=0)


vehicle_id                 38113
year                       38113
make                       38113
model                      38113
class                      38113
                           ...  
hours_to_charge_240v         175
hours_to_charge_ac_240v       46
composite_city_mpg            65
composite_highway_mpg         65
composite_combined_mpg        65
Length: 61, dtype: int64

In [17]:
data.drop(list(data.filter(regex = 'ft2')), axis = 1, inplace = True) #Rimozione dei valori relativi al Fuel Type 2, in quanto la colonna corrispondente aveva solamente valori nulli ed è stata eliminata.

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 52 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [19]:
data.astype(bool).sum(axis=0)


vehicle_id                           38113
year                                 38113
make                                 38113
model                                38113
class                                38113
drive                                38113
transmission                         38113
transmission_type                    38113
engine_index                         25517
engine_descriptor                    38113
engine_cylinders                     38113
engine_displacement                  38112
turbocharger                         38113
fuel_type                            38113
fuel_type_1                          38113
city_mpg_ft1                         38113
city_gasoline_consumption_cd            14
city_electricity_consumption           198
city_utility_factor                     65
highway_mpg_ft1                      38113
highway_gasoline_consumption_cd         11
highway_electricity_consumption        198
highway_utility_factor                  65
unadjusted_

In [20]:
data.drop(list(data.filter(regex = '_cd')), axis = 1, inplace = True) #Rimozione dei valori relativi al consumo in modalità di esaurimento della carica per veicoli elettrici, in quanto la colonna corrispondente aveva un numero insufficiente di valori non-nulli.

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 48 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [22]:
data.astype(bool).sum(axis=0)


vehicle_id                           38113
year                                 38113
make                                 38113
model                                38113
class                                38113
drive                                38113
transmission                         38113
transmission_type                    38113
engine_index                         25517
engine_descriptor                    38113
engine_cylinders                     38113
engine_displacement                  38112
turbocharger                         38113
fuel_type                            38113
fuel_type_1                          38113
city_mpg_ft1                         38113
city_electricity_consumption           198
city_utility_factor                     65
highway_mpg_ft1                      38113
highway_electricity_consumption        198
highway_utility_factor                  65
unadjusted_city_mpg_ft1              38088
unadjusted_highway_mpg_ft1           38088
combined_mp

In [23]:
data.drop(list(data.filter(regex = 'hours_to_charge')), axis = 1, inplace = True) #Rimozione dei valori relativi alle ore di ricarica dei veicoli ibridi/elettrici, in quanto non rilevanti all'analisi da effettuare.

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 45 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [25]:
data.drop(list(data.filter(regex = 'composite')), axis = 1, inplace = True) #Rimozione dei valori relativi al consumo composto per veicoli ibridi, in quanto aventi numero insufficiente di valori non nulli.

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38113 entries, 0 to 38112
Data columns (total 42 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   vehicle_id                         38113 non-null  int64  
 1   year                               38113 non-null  int64  
 2   make                               38113 non-null  object 
 3   model                              38113 non-null  object 
 4   class                              38113 non-null  object 
 5   drive                              36924 non-null  object 
 6   transmission                       38102 non-null  object 
 7   transmission_type                  15045 non-null  object 
 8   engine_index                       38113 non-null  int64  
 9   engine_descriptor                  22693 non-null  object 
 10  engine_cylinders                   37977 non-null  float64
 11  engine_displacement                37979 non-null  flo

In [27]:
data.astype(bool).sum(axis=0)


vehicle_id                           38113
year                                 38113
make                                 38113
model                                38113
class                                38113
drive                                38113
transmission                         38113
transmission_type                    38113
engine_index                         25517
engine_descriptor                    38113
engine_cylinders                     38113
engine_displacement                  38112
turbocharger                         38113
fuel_type                            38113
fuel_type_1                          38113
city_mpg_ft1                         38113
city_electricity_consumption           198
city_utility_factor                     65
highway_mpg_ft1                      38113
highway_electricity_consumption        198
highway_utility_factor                  65
unadjusted_city_mpg_ft1              38088
unadjusted_highway_mpg_ft1           38088
combined_mp

In [28]:
data.drop(list(data.filter(regex = '_utility_factor')), axis = 1, inplace = True) #Rimozione dei valori relativi al rapporto tra miglia viaggiate in modalità elettrica e miglia totali, in quanto aventi numero insufficiente di valori non nulli e non pertinente all'analisi.

In [29]:
data.astype(bool).sum(axis = 0)

vehicle_id                           38113
year                                 38113
make                                 38113
model                                38113
class                                38113
drive                                38113
transmission                         38113
transmission_type                    38113
engine_index                         25517
engine_descriptor                    38113
engine_cylinders                     38113
engine_displacement                  38112
turbocharger                         38113
fuel_type                            38113
fuel_type_1                          38113
city_mpg_ft1                         38113
city_electricity_consumption           198
highway_mpg_ft1                      38113
highway_electricity_consumption        198
unadjusted_city_mpg_ft1              38088
unadjusted_highway_mpg_ft1           38088
combined_mpg_ft1                     38113
combined_electricity_consumption       198
annual_fuel

In [30]:
data.transmission.value_counts()

transmission
Automatic 4-Speed                   11042
Manual 5-Speed                       8323
Automatic 3-Speed                    3151
Automatic (S6)                       2684
Manual 6-Speed                       2448
Automatic 5-Speed                    2191
Manual 4-Speed                       1483
Automatic 6-Speed                    1447
Automatic (S8)                        977
Automatic (S5)                        827
Automatic (variable gear ratios)      702
Automatic 7-Speed                     675
Automatic (S7)                        270
Auto(AM-S7)                           266
Automatic 8-Speed                     259
Automatic (S4)                        233
Auto(AM7)                             166
Auto(AV-S6)                           153
Automatic (A1)                        124
Auto(AM6)                             120
Automatic 9-Speed                     105
Auto(AM-S6)                            87
Auto(AV-S7)                            80
Manual 3-Speed       

In [31]:
data.transmission = data.transmission.replace('Manual 5 Speed', 'Manual 5-Speed')
data.transmission = data.transmission.replace('Automatic (S5)', 'Automatic 5-Speed')
data.transmission = data.transmission.replace('Automatic (S4)', 'Automatic 4-Speed')
data.transmission = data.transmission.replace('Automatic (S6)', 'Automatic 6-Speed')

In [32]:
data.transmission.unique()

array(['Manual 5-Speed', 'Automatic 3-Speed', 'Manual 4-Speed',
       'Automatic 4-Speed', 'Manual 3-Speed', 'Manual 4-Speed Doubled',
       nan, 'Manual 6-Speed', 'Automatic (variable gear ratios)',
       'Automatic 5-Speed', 'Auto(L4)', 'Auto(L3)', 'Automatic (A1)',
       'Automatic 6-Speed', 'Automatic 7-Speed', 'Automatic (S7)',
       'Automatic (S8)', 'Automatic (AV)', 'Auto(AM7)', 'Automatic (AM5)',
       'Auto(AM6)', 'Automatic (AM6)', 'Automatic (A6)', 'Auto(AV-S7)',
       'Manual 7-Speed', 'Auto (AV-S6)', 'Auto (AV-S8)',
       'Automatic (AV-S6)', 'Auto(AV-S6)', 'Auto(AM5)', 'Auto (AV)',
       'Auto(AV-S8)', 'Automatic 8-Speed', 'Auto(AM-S6)', 'Auto(AM-S7)',
       'Automatic 6Speed', 'Automatic 9-Speed', 'Automatic (S9)',
       'Auto(AM-S8)', 'Auto(AM8)', 'Manual(M7)', 'Auto(AM-S9)',
       'Automatic (S10)'], dtype=object)

In [33]:
data["class"].unique()

array(['Minicompact Cars', 'Two Seaters', 'Special Purpose Vehicle 2WD',
       'Special Purpose Vehicle 4WD', 'Subcompact Cars', 'Midsize Cars',
       'Midsize Station Wagons', 'Compact Cars',
       'Midsize-Large Station Wagons', 'Large Cars',
       'Small Station Wagons', 'Standard Pickup Trucks 2WD',
       'Vans, Passenger Type', 'Vans, Cargo Type',
       'Standard Pickup Trucks 4WD', 'Special Purpose Vehicles',
       'Small Pickup Trucks 2WD', 'Small Pickup Trucks 4WD', 'Vans',
       'Standard Pickup Trucks', 'Small Pickup Trucks', 'Vans Passenger',
       'Standard Pickup Trucks/2wd', 'Special Purpose Vehicles/2wd',
       'Special Purpose Vehicles/4wd', 'Sport Utility Vehicle - 4WD',
       'Sport Utility Vehicle - 2WD', 'Minivan - 2WD', 'Minivan - 4WD',
       'Special Purpose Vehicle', 'Small Sport Utility Vehicle 4WD',
       'Small Sport Utility Vehicle 2WD',
       'Standard Sport Utility Vehicle 4WD',
       'Standard Sport Utility Vehicle 2WD'], dtype=object)

In [34]:
data["class"].value_counts()

class
Compact Cars                          5508
Subcompact Cars                       4872
Midsize Cars                          4395
Standard Pickup Trucks                2354
Sport Utility Vehicle - 4WD           2082
Large Cars                            1891
Two Seaters                           1886
Sport Utility Vehicle - 2WD           1627
Small Station Wagons                  1499
Special Purpose Vehicles              1455
Minicompact Cars                      1260
Standard Pickup Trucks 2WD            1177
Vans                                  1141
Standard Pickup Trucks 4WD             986
Midsize-Large Station Wagons           656
Special Purpose Vehicle 2WD            613
Small Pickup Trucks                    538
Small Sport Utility Vehicle 4WD        526
Midsize Station Wagons                 523
Vans, Cargo Type                       438
Small Pickup Trucks 2WD                436
Standard Sport Utility Vehicle 4WD     434
Small Sport Utility Vehicle 2WD        403
Miniv

In [35]:
data["class"] = data["class"].replace('Vans Passenger', 'Vans, Passenger Type')
data["class"] = data["class"].replace('Standard Pickup Trucks/2wd', 'Standard Pickup Trucks 2WD')
data["class"] = data["class"].replace('Special Purpose Vehicles/2wd', 'Special Purpose Vehicle 2WD')
data["class"] = data["class"].replace('Special Purpose Vehicles/4wd', 'Special Purpose Vehicle 4WD')

In [36]:
data["class"].value_counts()

class
Compact Cars                          5508
Subcompact Cars                       4872
Midsize Cars                          4395
Standard Pickup Trucks                2354
Sport Utility Vehicle - 4WD           2082
Large Cars                            1891
Two Seaters                           1886
Sport Utility Vehicle - 2WD           1627
Small Station Wagons                  1499
Special Purpose Vehicles              1455
Minicompact Cars                      1260
Standard Pickup Trucks 2WD            1181
Vans                                  1141
Standard Pickup Trucks 4WD             986
Midsize-Large Station Wagons           656
Special Purpose Vehicle 2WD            615
Small Pickup Trucks                    538
Small Sport Utility Vehicle 4WD        526
Midsize Station Wagons                 523
Vans, Cargo Type                       438
Small Pickup Trucks 2WD                436
Standard Sport Utility Vehicle 4WD     434
Small Sport Utility Vehicle 2WD        403
Miniv

In [37]:
data.to_csv("fuel_cleaned.csv", index = False)